In [1]:
# Imports
import os
import asyncio
from typing import cast
from dotenv import load_dotenv
from typing import Dict
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI
from openai.types.responses import ResponseTextDeltaEvent


# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [2]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [3]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_gemma3 = env_to_str('MODEL_GEMMA3')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [4]:
# Agent Workflow Instructions

instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenix",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [6]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_gemma3,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [7]:
# Agents

sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=ollama_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=ollama_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=ollama_model
)

In [ ]:
# Phoenix Tracer
async def phoenix_tracer(trace_name, message, *agents):
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span(trace_name):
        if agents:
            for index, agent in enumerate(agents):
                # print(f"  Argument {index+1}: {agent}")
        
                # # Trace Attributes
                # current_span = otel_trace.get_current_span()
                # current_span.set_attribute("user.request", message)
                # current_span.set_attribute("agent.name", agent.name)
                # current_span.set_attribute("model.name", model_gemma3)
                results = await asyncio.gather(
                    Runner.run(agent, message)
                )
        else:
            print("No additional arguments provided.")

        outputs = [result.final_output for result in results]
        # result = await Runner.run(agents, message)
        
        # # Add response attributes
        # current_span.set_attribute("response.length", len(result.final_output))
        # current_span.set_attribute("response.preview", result.final_output[:100])
        for output in outputs:
            display(Markdown(output))
        return outputs
 

In [9]:
result = await phoenix_tracer(
    "Parallel Cold Emails", 
    "Write a cold sales email.", 
    sales_agent1, 
    sales_agent2, 
    sales_agent3
)

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


Okay, here's a cold email draft for ComplAI, focused on brevity and a direct value proposition. I've included a few options, ranging from more formal to slightly more conversational, so you can choose the one that best aligns with your target audience. **Please read the notes at the very end - they're crucial for success!**

**Option 1: Formal & Direct**

Subject: Simplify SOC2 Compliance with ComplAI

Dear [Prospect Name],

SOC2 audits are complex and time-consuming. ComplAI streamlines the process with AI-powered automation for documentation, evidence gathering, and controls testing.

Reduce audit preparation time and cost.

Learn more: [Link to ComplAI Website - typically a specific landing page]

Sincerely,

[Your Name]
[Your Title]
ComplAI

**Option 2: Slightly More Conversational**

Subject: Feeling overwhelmed by SOC2?

Hi [Prospect Name],

SOC2 compliance shouldn't be a burden. ComplAI uses AI to automate much of the heavy lifting - controls, documentation, evidence.

Imagine significantly reducing audit prep time and risk.

Check it out: [Link to ComplAI Website - typically a specific landing page]

Best,

[Your Name]
[Your Title]
ComplAI

**Option 3: Focused on a Specific Pain Point (Requires knowing a bit about the prospect)**

Subject: Streamline Your [Specific Control Area - e.g., Access Control] for SOC2

Dear [Prospect Name],

Is managing your [Specific Control Area] for SOC2 audit a constant challenge?

ComplAI uses AI to automate evidence collection and testing for controls like these, saving you time and ensuring accuracy.

See how: [Link to ComplAI Website - specifically a relevant feature page]

Regards,

[Your Name]
[Your Title]
ComplAI

**Important Notes & Best Practices (READ THIS!)**

*   **Personalization is Key:** These are templates.  *Always* personalize the email even a little. Mention their company, a blog post they published, or something relevant you found about their security posture (if possible – be careful about appearing intrusive).
*   **Targeting:** These emails *will not* work if sent to completely random people. Identify the right decision-makers (CISOs, Security Managers, Compliance Officers, IT Directors) and tailor the messaging accordingly.
*   **Subject Lines:** Test different subject lines! "SOC2 Compliance" is very generic. Consider questions or more direct value propositions.
*   **Landing Page:**  Direct the prospect to a *specific* landing page on your website.  Don’t just send them to the homepage.  A page focused on SOC2, free demo sign-up, or a case study is much more effective.
*   **Call to Action (CTA):** Make the CTA clear and concise. "Learn More" is okay, but "Schedule a Demo" or "Download Our SOC2 Checklist" can be stronger.
*   **Follow-Up:**  Most cold emails require follow-up.  A short, polite "Just checking in..." email can make a big difference.
*   **Compliance:** Be mindful of email marketing regulations (like GDPR and CAN-SPAM).  Make sure you have a legitimate reason to contact them.
*   **A/B Test:** Try different variations of your email (subject lines, body copy, CTAs) to see what performs best.



To help me tailor the email even better, can you tell me:

*   Who is your ideal customer profile (size of company, industry, etc.)?
*   What is the biggest pain point you hear from prospects regarding SOC2 compliance?

Exception while exporting Span.
Traceback (most recent call last):
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/luissantiago/Projects/ai-learning/agentic/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  Fi

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: ollama_o*********ault. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}
